Needs 3 text files with data, dataset.py, dataloader.py, and models (feedforward.py)

In [1]:
! pip install pytorch_lightning
! pip install torchmetrics

     |████████████████████████████████| 527 kB 5.1 MB/s 
     |████████████████████████████████| 397 kB 38.6 MB/s 
     |████████████████████████████████| 133 kB 51.4 MB/s 
     |████████████████████████████████| 952 kB 21.6 MB/s 
     |████████████████████████████████| 829 kB 38.8 MB/s 
     |████████████████████████████████| 596 kB 30.5 MB/s 
     |████████████████████████████████| 1.1 MB 36.6 MB/s 
     |████████████████████████████████| 271 kB 56.2 MB/s 
     |████████████████████████████████| 144 kB 53.8 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=1152d8e86ba33ce13a5958fabd9a3942cf9ebe50d51e7a47c4240d864fce8154
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successful

In [15]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from dataset import wiki_dataset
from dataloader import wiki_dataloader
from feedforward import FeedForward 
import torchmetrics
import pytorch_lightning as pl
import pytorch_lightning.loggers as pl_loggers
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
  # Load datasets
  train = wiki_dataset('wiki.train.txt', training=True, token_map='create')
  valid = wiki_dataset('wiki.valid.txt', training=False, token_map=train.token_map)
  test = wiki_dataset('wiki.test.txt', training=False, token_map=train.token_map)
  datasets = [train,valid,test]

  # Load dataloader
  dataloader = wiki_dataloader(datasets=datasets, batch_size=100)


In [ ]:
  # Make model and train
  model = FeedForward(context=train.window,embed_dim=100,vocab_size=len(train.unique_tokens))
  tb_logger = pl_loggers.TensorBoardLogger("./lightning_logs/", name="ff")
  trainer = pl.Trainer(logger=tb_logger, max_epochs=10, gpus=1)
  trainer.fit(model, dataloader)
  result = trainer.test(model, dataloader)
  print(result)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:470: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | embed | Embedding        | 2.9 M 
1 | lin1  | Linear           | 501 K 
2 | lin2  | Linear           | 28.9 M
3 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
32.3 M    Trainable params
0         Non-trainable params
32.3 M    Total params
129.293   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs/

In [ ]:
!ls /content/lightning_logs/staged_ff

In [ ]:
!rm -rf ./lightning_logs/